### 쇼핑몰 크롤링
- 상품의 시세 파악을 위해서는 쇼핑몰 데이터를 추출하여 파악하는 것이 일반적임
- 특정 사이트를 선정하여 매일 상품의 가격 변환를 확인
- 사이트 선정 시 구성이 변하지 않고, 접속량이 일정 수준 이상인 사이트를  선정
- http://jolse.com/category/toners-mists/1019/page=1

#### 크롤링 작업 순서
(1) 첫 번째 페이지에서 상품 정보 추출   
    - 상품 40개  
    - 각 상품박스  
        - 상품명  
        - 상품가격   
            - 정상가  
            - 세일가  

(2) 여러 페이지 크롤링  
    - (1) 접속 & 파싱 함수  
    - (2) 1개의 상품 추출 함수  
    - (3) 한 페이지에서 상품 추출  
    - (4) 여러 페이지에서 추출  
        - base_url + page 번호(i)  

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [46]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [4]:
# http://jolse.com/category/toners-mists/1019/page=1
# F12 개발자 도구에서 확인


In [ ]:
url = 'http://jolse.com/category/toners-mists/1019/page=1'
html = urlopen(url)
# 파서 객체 생성
bs_obj = BeautifulSoup(html, 'html.parser') 

In [17]:
# 페이지 구조
# 추출할 content만 담고 있는 div(class:description)영역을 활용해서 추출
ul = bs_obj.find("ul",{"class":"prdList grid5"})
# ul
lis = ul.find_all("li")
len(lis)
lis = ul.select("li")
len(lis)
lis = bs_obj.select("ul.prdList.grid5 li")
len(lis)

120

120

120

In [18]:
# 첫 페이지에서 상품 정보 추출
# 상품 정보만 담고 있는 div 태그 사용 (상품 이미지 제외)
boxes = bs_obj.find_all("div", {"class":"description"})
len(boxes)

40

In [19]:
# 추출할 요소 파악
boxes[10].find("strong", {"class":"name"})
boxes[10].find("strong", {"class":"name"}).text
boxes[10].find("strong", {"class":"name"}).text.split(" : ")[1]

<strong class="name"><a class="" href="/product/skinfood-royal-honey-propolis-enrich-cream-mist-120ml/46912/category/1019/display/1/"><span class="title displaynone"><span style="font-size:14px;color:#555555;font-weight:bold;">Product Name</span> :</span> <span style="font-size:14px;color:#555555;font-weight:bold;">SKINFOOD Royal Honey Propolis Enrich Cream Mist 120ml</span></a></strong>

'Product Name : SKINFOOD Royal Honey Propolis Enrich Cream Mist 120ml'

'SKINFOOD Royal Honey Propolis Enrich Cream Mist 120ml'

#### (1) 상품명 추출

In [ ]:
# 전체 상품명 출력
for box in boxes:
    name = box.find("strong", {"class":"name"}).text.split(" : ")[1]

In [23]:
# 상품명을 리스트로 저장


#### (2) 상품 가격 추출

In [38]:
# 정상 가격 / 할인 가격 추출
len(boxes[5].ul.find_all("span")[1])
boxes[5].ul.find_all("span")[1].text
boxes[5].ul.find_all("span")[-1].text
boxes[5].ul.find_all("span")[-2].text.split("(")[0].strip()

1

'USD 21.00'

'(30% OFF)'

'USD 14.70'

In [ ]:
#########################################################################################

### (1) 첫 페이지의 전체 상품 정보 추출

In [43]:
# 첫 페이지의 전체 상품 정보 추출
# 상품명, 정상가격, 세일가격
for box in boxes:
    name = box.find("strong", {"class":"name"}).text.split(" : ")[1]
    price = box.ul.find_all("span")[1].text
    sale_price = box.ul.find_all("span")[-2].text.split("(")[0].strip()
    # print("제품평 :", name, "가격 :", price, "할인 가격 :", sale_price)

In [ ]:
##################################################################

#### 첫 페이지에서 상품 정보 추출하여 df로 저장
(1) 접속 / 파싱  
(2) 수집  
(3) df로 저장  

In [58]:
# (1) 접속 / 파싱
url = 'http://jolse.com/category/toners-mists/1019/page=1'
html = urlopen(url)
# 파서 객체 생성
bs_obj = BeautifulSoup(html, 'html.parser') 

In [44]:
# (2) 수집

# 빈 리스트  생성
prd_list=[]
price_list=[]
sale_price_list=[]

# 전체 상품 데이터 추출 : boxes 추출
boxes = bs_obj.find_all("div", {"class":"description"})

# 각 box에서 상품명, 정상가, 세일가 추출해서 리스트에 저장
for box in boxes:
    prd_list.append(box.find("strong", {"class":"name"}).text.split(" : ")[1])
    price_list.append(box.ul.find_all("span")[1].text)
    sale_price_list.append(box.ul.find_all("span")[-2].text.split("(")[0].strip())

In [45]:
# (3) df로 저장
# 각 리스트를 데이터프레임으로 생성
product_df = pd.DataFrame({"품목":prd_list,
                           "가격":price_list,
                           "할인가격":sale_price_list})
product_df

,품목,가격,할인가격
0,SKIN1004 Madagascar Centella Toning Toner 210ml,USD 20.00,USD 14.80
1,COSRX Full Fit Propolis Synergy Toner 150ml,USD 28.00,USD 14.00
2,ROUND LAB 1025 Dokdo Toner 200ml,USD 17.00,USD 12.29
3,SKIN1004 Madagascar Centella Probio-Cica Essen...,USD 24.00,USD 14.88
4,numbuzin No.5 Vitamin-Niacinamide Concentrated...,USD 26.00,USD 20.80
5,SKIN1004 Madagascar Centella Poremizing Clear ...,USD 21.00,USD 14.70
6,celimax Jiwoogae Heartleaf BHA Peeling Pad 60p...,USD 17.50,USD 12.25
7,haruharu wonder Black Rice Hyaluronic Toner 15...,USD 22.00,USD 15.40
8,COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml,USD 17.25,USD 12.76
9,ROUND LAB 1025 Dokdo Toner 100ml,USD 13.00,USD 8.79


In [ ]:
######################### 첫 페이지(한 페이지)에서 상품 정보 추출 완료

### (2) 여러 페이지 크롤링
- 현재 페이지 뿐 아니라 다른 모든 페이지에서도 반복 적용할 수 있도록 함수로 작성

####  함수명 및 수행 기능   
(1) get_request_product(url) : 접속 및 파싱 함수   
    - url 받아서 접속 및 파싱  
    - 반환값 : bs4 객체  
(2) get_product_info(box) : 1개 상품 정보 추출 함수  
    - box 받아서 상품 정보 추출 후 반환  
    - 반환값 : 1개 상품의 상품명/가격/세일가격을 딕셔너리 형태로 반환  
(3) get_page_product(url) : 각 페이지에서 전체 상품 정보 추출하고 df에 추가 함수  
    - 전달받은 url 페이지에서 box 추출하여  
    - get_product_info(box) 호출해서 전달하고 반환된 1개의 상품 정보로  
    - 데이터프레임 생성 및 추가  

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
# 최종 상품 정보 저장할 빈 데이터프레임 생성
all_product_df = pd.DataFrame({"품목":[],
                               "가격":[],
                               "할인가격":[]})

In [41]:
# (1) 접속 및 파싱 기능 수행하는 함수
def get_request_product(url):
    try :
        # url 받아서 접속 및 파싱
        html = urlopen(url)
        bs_obj = BeautifulSoup(html, 'html.parser') 
    except Exception as e:
        print("오류 발생", e)
        
    # bs4 객체 반환
    return bs_obj

In [42]:
# (2) 1개의 상품 정보를 추출하는 함수
def get_product_info(box):
    try:
        # 1개 상품 정보 추출
        # box 받아서 상품 정보 추출 후 반환
        name = box.find("strong", {"class":"name"}).text.split(" : ")[1]
        price = box.ul.find_all("span")[1].text
        sale_price = box.ul.find_all("span")[-2].text.split("(")[0].strip()
    except Exception as e:
        print("오류 발생", e)

    # 반환값: 1개 상품의 상품명/가격/할인가격을 딕셔너리 형태로 반환
    return {"품목":name, "가격":price, "할인가격":sale_price}

In [57]:
# (3) 전달 받은 url 한 페이지에서 상품 정보 추출하고 df에 추가하는 함수
def get_page_product(url):
    global all_product_df # 함수 외부에서 정의된 변수를 인지하도록 할 때 쓰는 예약어
    # print(url)
    try:
        bs_obj=get_request_product(url)

        boxes=bs_obj.find_all("div", {"class":"description"})
    except Exception as e:
        print("오류 발생", e)

    for box in boxes:
        temp_df = pd.DataFrame(get_product_info(box), index=range(1, 2))
        all_product_df = pd.concat([all_product_df, temp_df], axis=0, ignore_index=True)

In [51]:
# 함수 테스트 : 테스트 후  주석 처리할 것

url = 'http://jolse.com/category/toners-mists/1019/page=2'
get_page_product(url)
# all_product_df.tail()
# ---------------------------------------------------------------------------
# UnboundLocalError                         Traceback (most recent call last)
# Cell In[65], line 2
#       1 url = 'http://jolse.com/category/toners-mists/1019/page=2'
# ----> 2 get_page_product(url)
#       3 all_product_df
# Cell In[64], line 12
#      10 for box in boxes:
#      11     temp_df = pd.DataFrame(get_product_info(box), index=range(1, 2))
# ---> 12     all_product_df = pd.concat([all_product_df, temp_df], axis=0, ignore_index=True)

# UnboundLocalError: cannot access local variable 'all_product_df' where it is not associated with a value

# 해당 변수가 만들어지지 않았거나 함수 밖에서 만들어져서 인식을 못하는 경우에 생기는 에러
# 이런 경우에는 global 예약어를 통해서 함수에 전달해줘야함 or 인수로 전달 or 함수 내부에서 변수 선언

http://jolse.com/category/toners-mists/1019/page=2


,품목,가격,할인가격
0,SKIN1004 Madagascar Centella Toning Toner 210ml,USD 20.00,USD 13.99
1,COSRX Full Fit Propolis Synergy Toner 150ml,USD 28.00,USD 13.29
2,ROUND LAB 1025 Dokdo Toner 200ml,USD 17.00,USD 13.60
3,SKIN1004 Madagascar Centella Probio-Cica Essen...,USD 24.00,USD 13.99
4,numbuzin No.5 Vitamin-Niacinamide Concentrated...,USD 26.00,USD 20.80
...,...,...,...
2095,COSRX CENTELLA WATER ALCOHOL FREE TONER 150ml,USD 17.25,USD 12.39
2096,AXIS-Y The Mini Glow Set,USD 9.80,USD 7.84
2097,numbuzin No.3 Super Glowing Essence Toner 100ml,USD 13.00,USD 10.40
2098,COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml,USD 17.81,USD 11.59


In [26]:
# 상품 저장 결과 확인
# 결과 확인 후
# 데이터프레임을 다시 초기화 할 것

In [27]:
################# 함수 잘 수행되는 것 확인했음

#### 여러 페이지 추출  
- url 뒤에 페이지 번호 있음  
- url 작성 시 페이지 번호를 따로 붙임   
- 'http://jolse.com/category/toners-mists/1019/page=' + 페이지 번호  
    - url 중간에 ? : get method 요청
    - url?전송데이터 : 뒤쪽의 문자열은 서버에게 보내는 정보 : page

In [44]:
# 마지막 페이지 수 추출
url = 'https://jolse.com/category/toners-mists/1019/page=1?page=1'
html = urlopen(url)
# 파서 객체 생성
bs_obj = get_request_product(url)

In [45]:
last_page = bs_obj.find("a", {"class":"last"})["href"].split("=")[1]
type(last_page)

str

### 모든 페이지에서 상품 정보 추출

In [58]:
# 최종 상품 정보 저장할 빈 데이터프레임 생성
all_product_df = pd.DataFrame({"품목":[],
                               "가격":[],
                               "할인가격":[]})

In [59]:
base_url = 'https://jolse.com/category/toners-mists/1019/page=1?page='

for page in range(1, int(last_page)+1):
    url = base_url+str(page)
    get_page_product(url)

In [62]:
all_product_df.tail()
# df 파일로 저장
all_product_df.to_csv("./crawl_data/shopping_product.csv", index=0, encoding="utf-8-sig")

,품목,가격,할인가격
500,preange Angelica Toner Quick Calming Pad 5ea,USD 12.00,USD 10.20
501,MAY ISLAND 7 Days Secret Vita Plus-10 Toner 155ml,USD 16.50,USD 14.02
502,Wish Formula Fermented Hydra Essential Toner 1...,USD 25.26,USD 19.20
503,BLITHE VItal Treatment 9 Essential Seeds 150ml,USD 40.80,USD 34.68
504,su:m37 LosecSumma Elixir Skin Softner 150ml,USD 110.00,USD 93.50


In [64]:
# 파일에서 읽어오기 확인
all_product_df = pd.read_csv("./crawl_data/shopping_product.csv")
all_product_df

,품목,가격,할인가격
0,SKIN1004 Madagascar Centella Toning Toner 210ml,USD 20.00,USD 13.99
1,COSRX Full Fit Propolis Synergy Toner 150ml,USD 28.00,USD 13.29
2,ROUND LAB 1025 Dokdo Toner 200ml,USD 17.00,USD 13.60
3,SKIN1004 Madagascar Centella Probio-Cica Essen...,USD 24.00,USD 13.99
4,numbuzin No.5 Vitamin-Niacinamide Concentrated...,USD 26.00,USD 20.80
...,...,...,...
500,preange Angelica Toner Quick Calming Pad 5ea,USD 12.00,USD 10.20
501,MAY ISLAND 7 Days Secret Vita Plus-10 Toner 155ml,USD 16.50,USD 14.02
502,Wish Formula Fermented Hydra Essential Toner 1...,USD 25.26,USD 19.20
503,BLITHE VItal Treatment 9 Essential Seeds 150ml,USD 40.80,USD 34.68
